# Manager report

In [3]:
import duckdb

duckdb_path = "data/sakila.duckdb"

dfs = {}

with duckdb.connect(duckdb_path) as conn:
    description = conn.sql("DESC;").df()

    for name in description["name"]:
        if name[0] != '_':
            dfs[name] = conn.sql(f"FROM staging.{name};").df()
            duckdb.register(name, dfs[name])
    
duckdb.sql("DESC;").df()

,database,schema,name,column_names,column_types,temporary
0,temp,main,actor,"[actor_id, first_name, last_name, last_update,...","[DOUBLE, VARCHAR, VARCHAR, TIMESTAMP, VARCHAR,...",True
1,temp,main,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, ""NULL"", VARCHAR, BIGINT, VAR...",True
2,temp,main,category,"[category_id, name, last_update, _dlt_load_id,...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",True
3,temp,main,city,"[city_id, city, country_id, last_update, _dlt_...","[BIGINT, VARCHAR, BIGINT, TIMESTAMP, VARCHAR, ...",True
4,temp,main,country,"[country_id, country, last_update, _dlt_load_i...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",True
5,temp,main,customer,"[customer_id, store_id, first_name, last_name,...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, BI...",True
6,temp,main,film,"[film_id, title, description, release_year, la...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, BI...",True
7,temp,main,film_actor,"[actor_id, film_id, last_update, _dlt_load_id,...","[BIGINT, BIGINT, TIMESTAMP, VARCHAR, VARCHAR]",True
8,temp,main,film_category,"[film_id, category_id, last_update, _dlt_load_...","[BIGINT, BIGINT, TIMESTAMP, VARCHAR, VARCHAR]",True
9,temp,main,inventory,"[inventory_id, film_id, store_id, last_update,...","[BIGINT, BIGINT, BIGINT, TIMESTAMP, VARCHAR, V...",True


1. a) Which movies are longer than 3 hours (180 minutes), show the title and its length?

In [4]:
film_len180 = duckdb.sql("""
    SELECT
        title,
        length
    FROM
        film
    WHERE
        length > 180;
""").df()


film_len180.index = film_len180.index + 1
film_len180

,title,length
1,ANALYZE HOOSIERS,181
2,BAKED CLEOPATRA,182
3,CATCH AMISTAD,183
4,CHICAGO NORTH,185
5,CONSPIRACY SPIRIT,184
6,CONTROL ANTHEM,185
7,CRYSTAL BREAKING,184
8,DARN FORRESTER,185
9,FRONTIER CABIN,183
10,GANGS PRIDE,185


1. b) Which movies have the word "love" in its title? Show the following columns
- Note: Only movies that have the word "love" is on the list. This means that movies with "lovely" will not be a part of the list.

In [7]:
import pandas as pd

pd.set_option("display.max_colwidth", None)

film_love = duckdb.sql("""
    SELECT
        title,
        rating,
        length,
        description
    FROM
        film
    WHERE
        title LIKE '% LOVE' OR
        title LIKE 'LOVE %' OR
        title LIKE '% LOVE %';
""").df()

film_love.index = film_love.index + 1
film_love

,title,rating,length,description
1,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And a Hunter who must Build a Composer in Berlin
2,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodile who must Meet a Database Administrator in The Outback
3,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad Scientist who must Kill a Astronaut in An Abandoned Fun House
4,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator And a Mad Cow who must Pursue a Womanizer in Berlin
5,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explorer who must Pursue a Dentist in An Abandoned Fun House


1. c) Calculate descriptive statistics on the length column

In [13]:
film_length = duckdb.sql("""
    SELECT
        MIN(length) AS Shortest_min,
        round(AVG(length),0)::INT AS Average_min,
        round(MEDIAN(length),0)::INT AS Median_min,
        MAX(length) AS Longest_min
    FROM
        film;
""").df()

film_length

,Shortest_min,Average_min,Median_min,Longest_min
0,46,115,114,185


1. d)  The 10 most expensive movies to rent per day

In [14]:

film_dayrent = duckdb.sql("""
    SELECT
        title,
        round((rental_rate / rental_duration),2) AS Rent_per_day
    FROM
        film
    ORDER BY
        Rent_per_day DESC,
        title ASC
    LIMIT
        10;
""").df()

film_dayrent.index = film_dayrent.index + 1
film_dayrent

,title,Rent_per_day
1,ACE GOLDFINGER,1.66
2,AMERICAN CIRCUS,1.66
3,AUTUMN CROW,1.66
4,BACKLASH UNDEFEATED,1.66
5,BEAST HUNCHBACK,1.66
6,BEHAVIOR RUNAWAY,1.66
7,BILKO ANONYMOUS,1.66
8,CARIBBEAN LIBERTY,1.66
9,CASPER DRAGONFLY,1.66
10,CASUALTIES ENCINO,1.66


1. e)  Which actors have played in most movies?

In [15]:
most_movies = duckdb.sql("""
    SELECT
        a.first_name || ' ' || a.last_name AS Actor,
        COUNT(*) AS Nr_movies
    FROM
        film_actor fa
    LEFT JOIN actor a ON a.actor_id = fa.actor_id
    GROUP BY
        Actor
    ORDER BY
        Nr_movies DESC
    LIMIT
        10;
""").df()

most_movies.index = most_movies.index + 1
most_movies

,Actor,Nr_movies
1,SUSAN DAVIS,54
2,GINA DEGENERES,42
3,WALTER TORN,41
4,MARY KEITEL,40
5,MATTHEW CARREY,39
6,SANDRA KILMER,37
7,SCARLETT DAMON,36
8,VAL BOLGER,35
9,VIVIEN BASINGER,35
10,ANGELA WITHERSPOON,35
